[N-rozměrná koule](https://en.wikipedia.org/wiki/Volume_of_an_n-ball)

## Určení obsahu jednotkového kruhu pomocí metody Monte Carlo.

Označme $S$ jednotkový kruh a $C$ jemu opsaný čtverec $[-1, 1]\times[-1, 1]$. 
Pokud vezmeme náhodný bod uvnitř $C$ (o obsahu $4$), je pravděpodobnost $p$, že bod bude uvnitř kruhu rovna
$$ p = \frac{|S|}{|C|} = \frac{|S|}{4} $$
Metoda Monete Carlo spočívá ve vygenerování velkého množství náhodných bodů a určení podílu těch, padly dovnitř kruhu. V tomto jednoduchém případě víme, že přesná hodnota je $$ p = \frac{\pi}{4} $$. 

In [26]:
import random
import numpy as np


def get_point():
    return [random.uniform(-1,1), random.uniform(-1,1)]


def is_in_disk(x, y):
    return x*x+y*y < 1


    

In [27]:
powers = [1, 2, 3, 4, 5, 6, 7]
sample_sizes = [10**i for i in powers]

def sampling(sample_sizes):
    n_samples = 0
    n_in = 0
    counts = []
    for size in sample_sizes:
        while n_samples < size:
            x, y = get_point()
            n_in += is_in_disk(x, y)
            n_samples+=1        
        counts.append(n_in)
    return counts
    


In [29]:
from timeit import default_timer as timer

begin = timer()
counts = sampling(sample_sizes)
end = timer()
print("Sampling time: ", end - begin)

Sampling time:  10.827210155985085


Zde je vidět poněkud slabý výpočetní výkon Pythonu, na jeden vzorek (sample) potřebujeme řádově 1000 taktů procesoru.


In [30]:
exact_pst = np.pi / 4    
sigma = (1 - exact_pst)*exact_pst
for power, size, count in zip(powers, sample_sizes, counts):
    approx = float(count)/size
    error = np.abs(approx - exact_surface)
    estim_error = sigma / np.sqrt(size)
    print(power, 4*approx, error, error / estim_error)


1 3.2 0.0146018366026 0.273958114476
2 3.0 0.0353981633974 2.1001843303
3 3.088 0.0133981633974 2.51374925067
4 3.1592 0.00440183660255 2.61162370302
5 3.14172 3.18366025517e-05 0.0597314970972
6 3.14296 0.000341836602552 2.02812747131
7 3.1416196 6.73660255168e-06 0.12639142481


In [ ]:
S rostoucím počtem vzorků nám klesá chyba, ale velmi pomalu s odmocninou z počtu vzorků, t.j. poslední sloupec je přibližně 1.

In [31]:
size = 10**7

begin = timer()
XY = np.random.rand(size, 2)
count  = np.sum( np.sum(XY**2, axis = -1) < 1.0 )
end = timer()
print("Sampling time: ", end - begin)
print(4*count/size)

Sampling time:  1.368625033996068
3.1419264


Pomocí použití Numpy sice převedeme většinu výpočtu do kompilované knihovny a dosáhneme asi 10x urychlení, 
ale zase je tento přístup zbytečně náročný na paměť a je limitován pomalým přístupem do paměti. 
To nyní pomineme a pokusíme se o urychlení pomocí paralelního zpracování.

Existují dva koncepty:

    1. Spuštění více procesů -- instancí stejného programu. Výhodou je úplné oddělení procesů, 
    nevýhodou N kopií programu v paměti a složitější komunikace mezi procesy.
    
    2. Spuštění více vláken (thread) v rámci jednoho programu. Výhodou je rychlejší spouštění vláken, 
    menší nároky na paměť a snadnější komunikace vláken, nevýhodou je složitější ladění a problémy s 
    globálními proměnnými. 
    
Efektivní využití dnešního HW vyžaduje použití obou přístupů:
    1. Výpočetní cluster je sestaven ze samostatných počítačů, tzv. uzlů. Na každém uzlu běží samostatný proces, 
    který komunikuje s procesy na jiných uzlech pomocí posílání zpráv (MPI viz. dále).
    
    2. V rámci jednoho uzlu si proces vytvoří více výpočetních vláken, aby využil všechna jádra uzlu.
    
    3. Optimalizace. Pro maximální využití výpočetních prostředků se provádějí optimalizace na správné využití cache a 
    vektorových výpočetních operací.

## Global Interpreter Lock (GIL)

Python umí používat vlákna, ale zároveň používá globální zámek, který praktický zajistí, že v každý okamžik aktuálně běží pouze jedno z existujících vláken. Zde se ukazuje, že Python (podobně jako mnoho dalšího SW) nebyl na vícevláknový běh navržen a pro zaručení konzistentního stavu všech interních operací to bylo vyřešeno globálním zámkem. Využití vláken v Pythonu tedy je vhodné pro:
- obsluhu asynchonních událostí (komunikace po síti, přerušení)
- zefektivnění operací (síť, disk) které neváznou na procesoru

Tento koncept se nazývá "concurency programming" narozdíl od "parallel programming", kde chceme aby více vláken nebo procesů běželo současně. Standardní knihvna pythonu poskytuje dva moduly s podobným rozhraním: modul `threading` pro 
práci s vlákny synchronyzovanými pomocí GIL a modul `multiprocessing` pro koordinaci více samostatných procesů. Je třeba upozornit, že GIL nevyužívají všechny implementace Pythonu.
- CPython - GIL, referenční implementace
- PyPy - GIL, just-in-time compiler (cca 5x zrychlení), mírná omezení oproti Pythonu
- Jython - no GIL, kompilace do JAVA byte kódu
- IronPython - no GIL, založeno na Microsoft .NET platformě

Tj. s použitím Jthonu nebo IronPythonu běží vlákna skutečně paralelně.

modul multiprocessing

Výpočet vzroků v metodě MC rozdělíme na více procesů, při vytvoření procesu mu předáme funkci kterou má spustit spolu s
jejími parametry. Parametry zahrnují i pole `counts` pro uložení výsledků.


In [14]:
import random
import numpy as np
import queue

def sampling(worker_id, size, counts):
    XY = np.random.rand(size, 2)
    count  = np.sum( np.sum(XY**2, axis = -1) < 1.0 )
    counts.put( (size, count) )
    print(worker_id, size, count)

def process_samples(counts):
    total_count = 0
    total_samples = 0
    while not counts.empty():
        n_samples, count = counts.get()
        total_samples += n_samples
        total_count += count
    print("pi:", 4*float(total_count)/total_samples)    
    

counts = queue.Queue()   
sampling(0, 10000, counts)
process_samples(counts)

0 10000 7855
pi: 3.142


In [19]:
import multiprocessing


    
    
n_samples = 10**7   
n_proc = 4
q = multiprocessing.Queue()
jobs = []
for i in range(n_proc):
    n_samples_local = int(n_samples / (n_proc - i))
    n_samples -= n_samples_local
    p = multiprocessing.Process(target=sampling, args=(i, n_samples_local, q))
    jobs.append(p)
    p.start()

    
# Wait for all processes to finish.
#q.close()
#q.join_thread()
for p in jobs:
    p.join()
    
process_samples(q)

3 2500000 1962976
1 2500000 1962976
2 2500000 1962976
0 2500000 1962976
pi: 3.1407616


Jelikož vytváření procesů je drahé, používá se tzv. Pool. Tedy pevná množina procesů, které se vytvoří jen jednou a pak se jim přiřazuje práce, kterou mají vykonat.